In [1]:
import numpy as np
import matplotlib.pyplot as plt


In [7]:
def A_dh(theta, d, a, alpha, type, radians = False):
    if not radians:
        theta = theta / 180 * np.pi
    if not radians:
        alpha = alpha / 180 * np.pi
    return np.array([
        [np.cos(theta), -np.sin(theta)*np.cos(alpha), np.sin(theta)*np.sin(alpha), a*np.cos(theta)],
        [np.sin(theta), np.cos(theta)*np.cos(alpha), -np.cos(theta)*np.sin(alpha), a*np.sin(theta)],
        [0, np.sin(alpha), np.cos(alpha), d],
        [0, 0, 0, 1]
    ])
    
def A_trans(x, y, z, type):
    return np.array([
        [1, 0, 0, x],
        [0, 1, 0, y],
        [0, 0, 1, z],
        [0, 0, 0, 1]
    ])

In [8]:
dh_conv = {
    1 : {'theta': 0, 'd': 50/1000, 'a': 0, 'alpha': 90, 'type': 'revolute'},
    2 : {'theta': 0, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    3 : {'theta': 0, 'd': 0, 'a': 93/1000, 'alpha': 0, 'type': 'revolute'},
    4 : {'theta': 0, 'd': 0, 'a': 50/1000, 'alpha': 0, 'type': 'revolute'},
    5: {'x' : -15/1000, 'y' : 45/1000, 'z' : 0, 'type': 'transformation'}
}



In [ ]:
A = {
    '01' : A_dh(**dh_conv[1]),
    '12' : A_dh(**dh_conv[2]),
    '23' : A_dh(**dh_conv[3]),
    '34' : A_dh(**dh_conv[4]),
    '45' : A_trans(**dh_conv[5])
}

In [ ]:
T = {
    '00' : np.eye(4),
    '01' : A['01'],
    '02' : A['01'] @ A['12'],
    '03' : A['01'] @ A['12'] @ A['23'],
    '04' : A['01'] @ A['12'] @ A['23'] @ A['34'],
    '05' : A['45'] @ A['01'] @ A['12'] @ A['23'] @ A['34']
}